In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tnguy70/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tnguy70/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe();

In [5]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name']))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['cozy', 'entire', 'floor', 'of', 'brownstone']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['cozy', 'entire', 'floor', 'brownstone']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

/software/anaconda3/5.3.0/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [13]:
# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.516*"bedroom" + 0.258*"clean" + 0.224*"good" + 0.000*"cozy" + '
  '0.000*"beautiful" + 0.000*"time" + 0.000*"bathroom" + 0.000*"train" + '
  '0.000*"new" + 0.000*"close"'),
 (1,
  '0.997*"private" + 0.000*"cozy" + 0.000*"beautiful" + 0.000*"bathroom" + '
  '0.000*"time" + 0.000*"train" + 0.000*"close" + 0.000*"new" + '
  '0.000*"renovate" + 0.000*"quiet"'),
 (2,
  '0.525*"comfortable" + 0.472*"sunny" + 0.000*"cozy" + 0.000*"beautiful" + '
  '0.000*"train" + 0.000*"time" + 0.000*"bathroom" + 0.000*"close" + '
  '0.000*"new" + 0.000*"renovate"'),
 (3,
  '0.404*"village" + 0.286*"huge" + 0.141*"loft" + 0.090*"historic" + '
  '0.077*"deck" + 0.000*"cozy" + 0.000*"beautiful" + 0.000*"time" + '
  '0.000*"train" + 0.000*"bathroom"'),
 (4,
  '0.808*"private" + 0.189*"brownstone" + 0.000*"cozy" + 0.000*"beautiful" + '
  '0.000*"bathroom" + 0.000*"time" + 0.000*"train" + 0.000*"close" + '
  '0.000*"new" + 0.000*"renovate"'),
 (5,
  '0.597*"large" + 0.400*"room" + 0.000*"cozy" + 0.000*"

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -17.611995808719314

Coherence Score:  0.21276131595658612


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/software/anaconda3/5.3.0/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.012283  0.158344       1        1  5.245407
7      0.012251  0.158206       2        1  5.110619
10     0.012233  0.158131       3        1  5.087787
9      0.016547  0.262241       4        1  5.049904
0      0.016528  0.262155       5        1  5.039217
3      0.012134  0.157701       6        1  5.020151
5      0.288167 -0.178739       7        1  5.018787
4     -0.360419 -0.159035       8        1  4.983241
15     0.012142  0.157730       9        1  4.973508
8      0.012175  0.157877      10        1  4.962577
12     0.015463  0.236121      11        1  4.960553
19     0.306062 -0.199703      12        1  4.958182
14    -0.357500 -0.156545      13        1  4.955298
6      0.321813 -0.220375      14        1  4.954349
16     0.321813 -0.220375      15        1  4.954348
11     0.321813 -0.220375      16        1  4.954347
1     -0.364666 -0.162362      17        1  4.950744
17    -0.364666 -0.162362      18        1  4.950742
18    -0.249664 -0.264895      19        1  4.950580
2      0.015491  0.236260      20        1  4.919664, topic_info=     Category           Freq         Term          Total  loglift  logprob
term                                                                      
60    Default  598873.000000      private  598873.000000  30.0000  30.0000
10    Default  541650.000000         room  541650.000000  29.0000  29.0000
24    Default  207468.000000      bedroom  207468.000000  28.0000  28.0000
6     Default  134666.000000       studio  134666.000000  27.0000  27.0000
5     Default  134425.000000     spacious  134425.000000  26.0000  26.0000
7     Default  118426.000000    apartment  118426.000000  25.0000  25.0000
38    Default  139620.000000        sunny  139620.000000  24.0000  24.0000
8     Default   81310.000000        large   81310.000000  23.0000  23.0000
70    Default   70070.000000  comfortable   70070.000000  22.0000  22.0000
40    Default   60613.000000          bed   60613.000000  21.0000  21.0000
66    Default   59454.000000     location   59454.000000  20.0000  20.0000
67    Default   57626.000000        great   57626.000000  19.0000  19.0000
73    Default   55793.000000        heart   55793.000000  18.0000  18.0000
69    Default   54993.000000      village   54993.000000  17.0000  17.0000
76    Default   48798.000000       modern   48798.000000  16.0000  16.0000
51    Default   46500.000000         home   46500.000000  15.0000  15.0000
47    Default   38985.000000         huge   38985.000000  14.0000  14.0000
3     Default   36542.000000        floor   36542.000000  13.0000  13.0000
108   Default   35340.000000        clean   35340.000000  12.0000  12.0000
13    Default   30627.000000         good   30627.000000  11.0000  11.0000
62    Default   29146.000000       bright   29146.000000  10.0000  10.0000
18    Default   27338.000000         side   27338.000000   9.0000   9.0000
0     Default   25511.000000   brownstone   25511.000000   8.0000   8.0000
95    Default   19246.000000         loft   19246.000000   7.0000   7.0000
129   Default   12279.000000     historic   12279.000000   6.0000   6.0000
71    Default   10432.000000         deck   10432.000000   5.0000   5.0000
22    Default    9898.000000       lovely    9898.000000   4.0000   4.0000
20    Default    9255.000000       garden    9255.000000   3.0000   3.0000
16    Default    8287.000000         east    8287.000000   2.0000   2.0000
19    Default    4679.000000         area    4679.000000   1.0000   1.0000
...       ...            ...          ...            ...      ...      ...
55    Topic20       0.152463    beautiful       2.466946   0.2281 -13.6829
133   Topic20       0.151676         stay       2.454224   0.2281 -13.6881
221   Topic20       0.151742        place       2.455316   0.2281 -13.6876
101   Topic20       0.151867        quiet       2.457349   0.2281 -13.6868
206   Topic20   